<a href="https://colab.research.google.com/github/briannabrownOE/FortuneCookies/blob/master/BB_Copy_fortune_cookie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from io import StringIO

import numpy as np
import pandas as pd
import requests
url='https://raw.githubusercontent.com/WomenInDataScience-Seattle/Machine_Learning_Projects/master/FortuneCookie/training_data/data.csv'
s=requests.get(url).text

c=pd.read_csv(StringIO(s))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.spatial.distance import cdist

In [0]:

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 

Using TensorFlow backend.


In [0]:
c.head(5)

,Fortune Cookie Quotes,Unnamed: 1
0,Be a generous friend and a fair enemy,NaN
1,Never quit!,NaN
2,"Old friends, old wines and old gold are best",NaN
3,"If your desires are not extravagant, they will...",NaN
4,Every Friend Joys in your Success,NaN


In [0]:
fortune_data = c['Fortune Cookie Quotes']

In [0]:
fortune_data.head(5)

0                Be a generous friend and a fair enemy
1                                         Never quit! 
2        Old friends, old wines and old gold are best 
3    If your desires are not extravagant, they will...
4                   Every Friend Joys in your Success 
Name: Fortune Cookie Quotes, dtype: object

In [0]:
fortune_data[1]

'Never quit! '

In [0]:
fortune_data[36]

'Let your heart make your decisions - it does not get as confused as your head. '

In [0]:
cleaned_df = fortune_data.str.lower()
cleaned_df2 = cleaned_df.str.strip()

In [0]:
dropped = cleaned_df2.dropna()

In [0]:
dropped.tail(5)

834    your present question marks are going to succeed.
835                your shoes will make you happy today.
836    your smile brings happiness to everyone you meet.
837    your way of doing what other people do their w...
838                            your wish will come true.
Name: Fortune Cookie Quotes, dtype: object

In [0]:
cleaned_fortunes = dropped

In [0]:
cleaned_fortunes.head(5)

0                be a generous friend and a fair enemy
1                                          never quit!
2         old friends, old wines and old gold are best
3    if your desires are not extravagant, they will...
4                    every friend joys in your success
Name: Fortune Cookie Quotes, dtype: object

In [0]:
cleaned_fortunes[3]

'if your desires are not extravagant, they will be granted'

In [0]:
cleaned_fortunes[0]

'be a generous friend and a fair enemy'

In [0]:
corpus = cleaned_fortunes

In [0]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[11, 5],
 [11, 5, 740],
 [11, 5, 740, 149],
 [11, 5, 740, 149, 9],
 [11, 5, 740, 149, 9, 5],
 [11, 5, 740, 149, 9, 5, 456],
 [11, 5, 740, 149, 9, 5, 456, 330],
 [22, 741],
 [137, 95],
 [137, 95, 137]]

In [0]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [0]:
predictors[60]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  43, 747, 219,
         9,  32], dtype=int32)

In [0]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 50, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(GRU(100, activation='relu'))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 41, 50)            86400     
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               45300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1728)              174528    
Total params: 306,228
Trainable params: 306,228
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(predictors, label, epochs=100, verbose=5)


Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [0]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            print(predicted)
            print(np.sum(predicted))
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()
  

In [0]:
def generate_text_prob(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_proba(token_list, verbose=0)
        random = np.random.choice(predicted.shape[1],1, p=predicted[0])
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == random:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()
  

In [0]:
token_list = tokenizer.texts_to_sequences('you')[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = model.predict_proba(token_list, verbose=0)
random = np.random.choice(predicted.shape[1],1, p=predicted[0])

print(random)
predicted[0].shape

[144]


(1728,)

In [0]:
!pip install random-word

In [0]:
from random_word import RandomWords

In [0]:
r = RandomWords()
random_word = r.get_random_word()

print(random_word)


canvassed


In [1]:
r = RandomWords()
random_word = 'Dreams'
text = generate_text_prob(random_word, 7, model, max_sequence_len)
print(text)

NameError: ignored

What we did today: 
- we changed to gru 
- we increased the word embedding length
- we increased the dropout
- we changed the activation from tanh to relu
- we randomly sampled our probaility distribution of word predictions

Next time:
- Use a pre-trained word embedding applied to our corpus
- get more data
- try training
